<a href="https://colab.research.google.com/github/npd2013/Mypublicmodels/blob/main/7_x_random_summary_generation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install needed packages 
#!pip install  gensim  #has word2vec model built, using english 

!pip install rouge
!pip install indic-nlp-library


In [2]:
from google.colab import files #upload the files from local machine on the colab environment 
uploaded = files.upload()


Saving story51.txt to story51.txt
Saving story51-gold.txt to story51-gold.txt
Saving story51-topic.txt to story51-topic.txt


In [3]:
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_preporcess-v6.py" "guj_final_preporcess.py" #The preprocessing modules designed
!cp "/content/drive/My Drive/Colab Notebooks/Evaluation.py" "Evaluation.py"   # the evaluation module designed


In [4]:
import io
import csv
from Evaluation import *
import random

import pandas as pd 
from guj_final_preporcess import * #to get the required preporcessing modules -from self made pipeline


In [5]:
#take all the needed inputs from user 
basename=input('Enter the basic name of file(without txt),other names wud be autogenerated  ')
filename=basename+'.txt'
gold_filename=basename+'-gold.txt'

summary_filename=basename+'-randomsummary.txt' # example file1-gold-system-summary.txt
analysefile_name=basename+'-analysis-randomsummary.txt'
rougescorefile="Rougeresults--randomsummarygeneration .csv"


Enter the basic name of file(without txt),other names wud be autogenerated  story51


In [31]:
#read the input original file . 

guj_clean_html(filename,filename)# text cleaning remove the html tags and overwrite the original file 
guj_clean_html(gold_filename,gold_filename)# clean the html tags from gold file 


guj_clean_extra(filename,filename)#remove the extra symbols like multiple punctuations and overwrite original file
guj_clean_extra(gold_filename,gold_filename)





#Display the given data in input files 
with io.open(filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file
analyzedfile_obj=open(analysefile_name,mode='w',encoding='UTF-8')
analyzedfile_obj.write("\n\n The original file is :------\n")
analyzedfile_obj.write(lines)

with io.open(gold_filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file

analyzedfile_obj.write("\n\n The gold summary provided by expert  is -->\n")    
analyzedfile_obj.write(lines)


###################################################################
########################  Step-1 ##################################
#####################  cleaning phase  ########################

#doing cleaning of file 
#remove newlines and other unnecessary data and clean it for feature extraction 
#to replace the data as per our format, replacing abbreviations dot with '*'
normfile="temp1.txt"
norm_gold_file="temp2.txt"


guj_clean_text(filename,normfile) 

guj_clean_text(gold_filename,norm_gold_file)

#Now display the stats and make corpora after cleaning file and removing new lines . 
normcorpora,totalnormsentences=guj_corpus_generate(normfile)  
goldcorpora,totalnormgoldsentences=guj_corpus_generate(norm_gold_file) 

originalcorpora,totaloriginalsentences=guj_corpus_generate(normfile)
sample='preprocessed.txt'
linenumber=1
with io.open(sample,'w',encoding='utf-8') as w: 
 for sent in originalcorpora:
   
     w.write(str(linenumber)+')')
     w.write(sent)
     w.write("\n")
     linenumber+=1 #increment line number when referening next line from corpora
originalcorpora,totaloriginalsentences=guj_corpus_generate(filename)

 


In [26]:
 rougescorefilehandle=open(rougescorefile,mode='a',encoding="utf-8") #to store only the rouge scores for later analysis 
 resultvar = csv.writer(rougescorefilehandle, lineterminator='\n')		#to store new row for each f  


################# ######Step4#############################
################ Find top sentences  #################

#rank and sort the sentences  based on individual sentences score 

 percent=(totalnormgoldsentences/totalnormsentences)*100 #ideal ratio as per gold text given 
 n=totalnormgoldsentences#extract sentences as per total sentences in gold summary 

 resultlist=[]#to store the rouge score data

 resultlist.append(filename+",") #mention the source file name for record
 
 
  ##########################################################
################# ######Step6#############################
################ Generate summary  #################
##########################################################
 summary_random_numbers=[] #store the random sentence numbers to be put in summary 

 summary_random_numbers=random.sample(range(1, totalnormsentences), n) #get n random numbers to map to sentence numbers  
 print(summary_random_numbers)
 linenumber=0

 with io.open(summary_filename,'w',encoding='utf-8') as w: 
  for sent in normcorpora:
   
   if linenumber  in summary_random_numbers:
     
     w.write(str(linenumber)+')')
     w.write(sent)
     w.write("\n")
   linenumber+=1 #increment line number when referening next line from corpora
     

 analyzedfile_obj.write("\n\n ***** The summary geneated has the following random numbered sentences ---->>\n {0}".format(summary_random_numbers))
 
  
norm_summary_filename="t.txt"
    
################ Evaluate summary  #################
 guj_remove_newlines(norm_gold_file,norm_gold_file)
 guj_remove_newlines(summary_filename,norm_summary_filename)
 rougescores=evalate_files(norm_gold_file,norm_summary_filename)#get the evaluation of generated summary after post processing based on rouge score
 for key in rougescores.keys():
     resultlist.append("%s,%s"%(key,rougescores[key]))
 resultvar.writerow(resultlist)   
 analyzedfile_obj.write("\n **********************Rouge scores based analysis for {0} % extraction is as follows*********************** ".format(percent))
 analyzedfile_obj.write("\n\n The rouge scores on summary file are \n {0}".format(rougescores,2)) #store the results 
 basicfile="original.txt"
  #guj_clean_text(filename,basicfile) # Get the original file in cleaned form for computing compression ratio. 
 guj_remove_newlines(filename,basicfile) 
  
 sent_compression,word_compression,totaloriginalsentences,totaloriginalwords,totalsumsentences,totalsumwords=compression_statistics(summary_filename,basicfile) #to store the compression ratio analysis
 analyzedfile_obj.write("\n **********************Compression Analysis at {0} % extraction is as follows(based on original file) *********************** ".format(percent))
 analyzedfile_obj.write("\n Total sentences in original text were {0} ".format(totaloriginalsentences)) 
 analyzedfile_obj.write("\n Total sentences in summary text are {0} ".format(totalsumsentences))
 analyzedfile_obj.write("\n Sentences compression ratio is {0} \n".format(sent_compression,2))
 analyzedfile_obj.write(" \n\n Total words in original text were {0} ".format(totaloriginalwords))
 analyzedfile_obj.write(" \n total words in summary text are {0} ".format(totalsumwords))
 analyzedfile_obj.write("\n Words compression ratio is {0} ".format(word_compression,2)) 



[32, 30, 8, 7, 10, 14, 25, 35, 1, 17, 19, 9, 5, 24, 31, 11, 18, 29, 26, 28, 36, 6, 3, 33]
sentence is એક વિજ્ઞાન શિક્ષક એમનું પૂરું નામ શૈલેષ કે ચૌહાણ.and sequence number is      0
sentence is ધાર્મિક સ્વભાવ ના માણસ.and sequence number is      1
sentence is kept in final summary 
sentence is એમનું મૂળ ગામ જૂનાગઢ જિલ્લા નું મેંદરડા.and sequence number is      2
sentence is ચૌહાણ સાહેબ જૂનાગઢ ની પુલકિત માધ્યમિક શાળા નામ ની ગ્રાન્ટેડ હાઈસ્કૂલ માં શિક્ષક અને તેમની પત્ની 
સરકારી પ્રાથમિક શાળા માં શિક્ષક.and sequence number is      3
sentence is kept in final summary 
sentence is શૈલેષ ના પિતા ડોક્ટર હતા પણ હવે ઉંમર વધી જવા ને કારણે હવે નિવૃત જીવન જીવી રહ્યા હતા.and sequence number is      4
sentence is શૈલેષ ના પિતા ને જૂનાગઢ માં ફાવતું ન હતું.and sequence number is      5
sentence is kept in final summary 
sentence is શૈલેષ ના પિતા એક પ્રકૃતિ પ્રેમી વ્યક્તિ હતા.and sequence number is      6
sentence is kept in final summary 
sentence is તેમની ઉંમર લગભગ પિંચોતેર વર્ષ ની આસપાસ હતી.and sequen

47

In [27]:
analyzedfile_obj.close()

rougescorefilehandle.close()

In [14]:
## code to download all data at end of execution .Uncomment it after all experiments done. 

from google.colab import files

!zip -r /content/txtfiles.zip *.txt
files.download("txtfiles.zip")
!zip -r /content/csvfiles.zip *.csv
files.download("csvfiles.zip")
!zip -r /content/codesfiles.zip *.py
files.download("codesfiles.zip")



updating: original.txt (deflated 75%)
updating: story51-analysis-randomsummary.txt (deflated 81%)
updating: story51-gold.txt (deflated 74%)
updating: story51-randomsummary.txt (deflated 73%)
updating: story51-topic.txt (stored 0%)
updating: story51.txt (deflated 75%)
updating: temp1.txt (deflated 75%)
updating: temp2.txt (deflated 74%)
updating: t.txt (deflated 73%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

updating: Rougeresults--randomsummarygeneration .csv (deflated 55%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

updating: Evaluation.py (deflated 64%)
updating: guj_final_preporcess.py (deflated 75%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>